In [1]:
!pip install pulp

In [19]:
import numpy as np
from pulp import LpMaximize, LpProblem, LpVariable, LpStatus, lpSum

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/dss/05_Варианты.csv", sep=';', index_col=0)
df.head()

,Ассортимент товара,Внешняя реклама,Внутренняя реклама,Коммуникабельность,Наличие автопарковки,Ассортимент конкурента 1,Ассортимент конкурента 2,Посещаемость в день (тыс. чел-к),Торговая площадь (тыс. кв м),Выручка за день (тыс. долларов)
Магазины,,,,,,,,,,
1,3,1,1,4,2,5,4,23.0,2.00,27.6
2,4,2,3,5,2,3,3,21.0,2.25,25.2
3,5,1,5,5,2,4,0,13.5,1.10,13.2
4,4,4,3,5,1,4,5,14.0,2.50,12.8
5,4,4,3,4,2,3,3,35.0,3.50,40.0


Выделим входы и выходы. К входам отнесем ассортимент, внешнюю и внутреннюю рекламу, коммуникабельность, наличие парковки и торговую площадь. Ассортименты конкурентов рассматривать не будем, так как это относится к внешним параметрам. В качестве выходов будем использовать посещаемость и выручку.

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 1 to 30
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Ассортимент товара                30 non-null     int64  
 1   Внешняя реклама                   30 non-null     int64  
 2   Внутренняя реклама                30 non-null     int64  
 3   Коммуникабельность                30 non-null     int64  
 4   Наличие автопарковки              30 non-null     int64  
 5   Ассортимент конкурента 1          30 non-null     int64  
 6   Ассортимент конкурента 2          30 non-null     int64  
 7   Посещаемость в день (тыс. чел-к)  30 non-null     float64
 8   Торговая площадь (тыс. кв м)      30 non-null     float64
 9   Выручка за день (тыс. долларов)   30 non-null     float64
dtypes: float64(3), int64(7)
memory usage: 2.6 KB


In [23]:
df.columns = ['assort', 'vnesh_rek', 'vnut_rek', 'komm', 'park', 'a1', 'a2', 'pos', 'plosch', 'vyr']

In [24]:
varV = [LpVariable(i, lowBound=0.00001) for i in df.columns[[0,1,2,3,4,8]] ]

In [25]:
varU = [LpVariable(i, lowBound=0.00001) for i in df.columns[[7, 9]]]

In [26]:
df.index

Int64Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
            18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
           dtype='int64', name='Магазины')

Создадим функцию, составляющую линеаризованную ЗЛП и решающую ее

In [27]:
def dea(mag, varV, varU):
  LP = LpProblem('dea_'+str(mag), sense=LpMaximize)

  LP += lpSum([df.loc[mag][var.name] *var for var in varV]) == 1, 'chislit'
  LP += lpSum([df.loc[mag][var.name] *var for var in varU])
  for i in df.index:
    if i != mag:
        LP += lpSum([df.loc[i][var.name] *var for var in varU]) -  lpSum([df.loc[i][var.name] *var for var in varV]) <= 0, i
  #print(LP)
  status = LP.solve()
  return LP

Для заданного списка магазинов mags расчитаем эффективность

In [28]:
mags = [1,3,5]
res = pd.DataFrame(index = mags, columns = ['assort', 'vnesh_rek', 'vnut_rek', 'komm', 'park', 'pos', 'plosch', 'vyr', 'efficiency'])
for i in mags:
  LP = dea(i, varV, varU)
  for var in LP.variables():
    res.loc[i][var.name] = var.value()
  res.loc[i]['efficiency'] = LP.objective.value()
res

,assort,vnesh_rek,vnut_rek,komm,park,pos,plosch,vyr,efficiency
1,1e-05,0.71397,0.00399696,1e-05,1e-05,1e-05,0.140971,0.0673491,1.85907
3,1e-05,0.260955,1e-05,1e-05,1e-05,0.0697473,0.671704,1e-05,0.941721
5,0.180968,1e-05,1e-05,1e-05,1e-05,0.0196198,0.0788569,0.00803135,1.00795
